In [11]:
from datasets import load_dataset

def formatData(data):
    if data == 'no':
        return 'A multi-line chart that does not suffer from overfitting.'
    elif 'overfitting after' in data and 'acc=' not in data:
        return f'A multi-line chart that shows {data}.'
    elif 'overfitting until' in data:
        return f'A multi-line chart that shows {data} neighbors.'
    elif ',acc' in data:
        if 'no' in data.split("nderfitting ")[0]:
            underfitting = 'no'
        else:
            underfitting = data.split("nderfitting ")[1].split(",")[0]
        
        if 'no' in data.split("overfitting ")[0]:
            overfitting = 'no'
        else:
            overfitting = data.split("overfitting ")[1].split(",")[0]
         
        if data.split("ac")[1] == 'c>80':
            acc = 'test accuracy line surpasses than 80%'
        elif data.split("ac")[1] == 'c<80':
            acc = 'test accuracy line does not surpass 80%'
        else:
            acc = 'test accuracy line stays at 80%'
        if underfitting == 'no' and overfitting == 'no':
            return f'A multi-line chart showing the random forest does not have underfitting, does not enter in overfitting and it\'s {acc}.'
        elif underfitting != 'no' and overfitting == 'no':
            return f'A multi-line chart showing the random forest is in underfitting until {underfitting} estimators, does not enter in overfitting and it\'s {acc}.'
        elif underfitting == 'no' and overfitting != 'no':
            return f'A multi-line chart showing the random forest does not have underfitting, enters in overfitting after {overfitting} estimators and it\'s {acc}.'
        else:
            return f'A multi-line chart showing the random forest is in underfitting until {underfitting} estimators, enters in overfitting after {overfitting} estimators and it\'s {acc}.'
    elif data == 'acc_rec':
        return 'A multi-line chart showing the train and test accuracy and recall of a decision tree.'
    elif 'samples' in data:
        return f'A decision tree with the following configuration:{data}.'
    elif data[0] == '[' and len(data.split(",")) > 2:
        return f'A bar chart showing the explained variance ration of each principal components the values are the following:{data}.'
    elif data[0] == '{' and data[1].isnumeric():
        return f'A correlation heatmap with {data[1:].split(",")[0]} features where the redundant pairs are the following:{data[:-1].split(",",1)[1]}.'
    elif data[0] == '[' and (data[1] == 'T' or data[1] == 'F') and data[2] == ',':
        return f'A set of boxplots that aren\'t normalized, in the following dictionary we can see each one of them and see if they have outliers and are balanced:{data[:-1].split(",",1)[1]}.'
    elif data[-1] == '}' and data[-2] == ']':
        return f'A set of bar charts for each symbolic variable:{data}.'
    elif data[0] == '{' and data[-2].isnumeric():
        return f'A bar chart showing the number of missing values for each variable:{data}.'
    elif data[0] == '[' and len(data.split(",")) == 2:
        return f'A bar chart with the following values:{data}.'
    elif 'Ordinal:' in data:
        return f'A set of histograms one for each variable, in the following dictionary we can see each one of them and see if they have outliers, if they are balanced and if it makes sense for them to be ordinal:{data}.'
    
    

def getData(chart):
    dataset = load_dataset('eduvedras/QAData',trust_remote_code=True)
    for i in range(len(dataset['train'])):
        if dataset['train'][i]['Chart_name'] == chart:
            return formatData(dataset['train'][i]['Data'])
    
    for i in range(len(dataset['test'])):
        if dataset['test'][i]['Chart_name'] == chart:
            return formatData(dataset['test'][i]['Data'])
    
    return 'ERRROR'

In [12]:
from tqdm.auto import tqdm
filetag="chartdatatrain1"
dataset = load_dataset('eduvedras/QA',split='train',trust_remote_code=True)

#file = open(f"{filetag}.jsonl", "a")
import json


with open(f"{filetag}.jsonl", 'w') as f:
    for i in tqdm(range(len(dataset))):
        chartdata = getData(dataset[i]['Chart_name'])
        question = dataset[i]['Question']
        answer = dataset[i]['Answer']
        line = {'messages':[]}
        line['messages'].append({"role": "system", "content": "You are a student doing a data science exam."})
        line['messages'].append({"role":"user", "content":f"Consider the following chart \"{chartdata}\", classify the following sentence about the chart as true or false: \"{question}\"."})
        line["messages"].append({'role':"assistant", "content":f"{'True' if answer=='T' else 'False'}"})
        f.write(json.dumps(line) +"\n")


100%|██████████| 2345/2345 [2:42:23<00:00,  4.16s/it]  
